In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
from sklearn.preprocessing import LabelEncoder

In [3]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

# Read the CSV and Perform Basic Data Cleaning

In [4]:
# https://help.lendingclub.com/hc/en-us/articles/215488038-What-do-the-different-Note-statuses-mean-

columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]

In [5]:
# Load the data
file_path = Path('LoanStats_2019Q1.csv')
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


In [6]:
#create copy of original dataframe
df2 = df.copy()

In [7]:
# use LabelEncoder from sklearn.preprocessing to convert columns with string datatypes to numeric values and replace same column
# note: pandas get_dummies does the same but is binary and creates new columns

le = LabelEncoder()

for column_name, item in df2.iteritems():
    
    if type(item) != int or float:
        
        df2[column_name] = le.fit_transform(df2[column_name])
        
    else:
        
        continue
        
df2.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,348,24,3500,3,2978,1,2,1,0,2590,...,285,155,0,0,11675,19018,19,27490,0,0
1,886,28,7846,1,4688,2,2,1,0,1889,...,331,101,1,0,39916,30368,412,21865,0,0
2,703,28,4973,1,2339,2,2,1,0,2292,...,142,101,0,0,10267,23239,75,18964,0,0
3,329,22,3251,3,4252,2,2,1,0,3010,...,409,101,1,0,19544,33595,197,32527,0,0
4,775,18,4894,1,2073,0,2,1,0,1742,...,409,0,0,0,34824,11407,276,6250,0,0


In [8]:
# check that data types are all numeric for all columns
df2_types_list = list(df2.dtypes)
# df2_types_list

# Split the Data into Training and Testing

In [9]:
# Create our features
X = df2.drop(columns = target)


# Create our target
y = df2["loan_status"]
y[:5]

0    1
1    1
2    1
3    1
4    1
Name: loan_status, dtype: int32

In [10]:
X.describe()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,pymnt_plan,dti,delinq_2yrs,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
count,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.0,68817.000000,68817.000000,...,68817.000000,68817.000000,68817.000000,68817.0,68817.000000,68817.000000,68817.000000,68817.000000,68817.0,68817.0
mean,572.296903,12.982577,4126.615168,1.812779,3294.087856,0.669994,0.805542,0.0,1962.376855,0.217722,...,367.820219,57.069314,0.125972,0.0,27710.412107,24184.046704,295.617696,19966.952715,0.0,0.0
std,375.779218,9.508701,2367.621789,0.941313,1574.576387,0.719105,0.714932,0.0,971.922012,0.717338,...,67.053040,55.878931,0.336732,0.0,17546.728857,14497.653994,257.292302,13661.709989,0.0,0.0
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0
25%,289.000000,5.000000,2153.000000,1.000000,1930.000000,0.000000,0.000000,0.0,1255.000000,0.000000,...,347.000000,0.000000,0.000000,0.0,12014.000000,11764.000000,116.000000,7895.000000,0.0,0.0
50%,516.000000,11.000000,3828.000000,1.000000,3365.000000,1.000000,1.000000,0.0,1842.000000,0.000000,...,409.000000,48.000000,0.000000,0.0,26735.000000,23056.000000,221.000000,18429.000000,0.0,0.0
75%,848.000000,20.000000,5967.000000,3.000000,4665.000000,1.000000,1.000000,0.0,2532.000000,0.000000,...,409.000000,101.000000,0.000000,0.0,42861.000000,36149.000000,393.000000,31243.000000,0.0,0.0
max,1397.000000,51.000000,8809.000000,3.000000,6089.000000,2.000000,2.000000,0.0,5695.000000,17.000000,...,409.000000,155.000000,4.000000,0.0,59628.000000,51997.000000,1715.000000,46898.000000,0.0,0.0


In [11]:
X.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,pymnt_plan,dti,delinq_2yrs,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,348,24,3500,3,2978,1,2,0,2590,0,...,285,155,0,0,11675,19018,19,27490,0,0
1,886,28,7846,1,4688,2,2,0,1889,0,...,331,101,1,0,39916,30368,412,21865,0,0
2,703,28,4973,1,2339,2,2,0,2292,0,...,142,101,0,0,10267,23239,75,18964,0,0
3,329,22,3251,3,4252,2,2,0,3010,0,...,409,101,1,0,19544,33595,197,32527,0,0
4,775,18,4894,1,2073,0,2,0,1742,0,...,409,0,0,0,34824,11407,276,6250,0,0


In [12]:
# Check the balance of our target values
Counter(y)

Counter({1: 68470, 0: 347})

In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1)

In [14]:
# check shapes of each training and testing set:
print(X_train.shape, X_test.shape,  y_train.shape, y_test.shape)

(51612, 85) (17205, 85) (51612,) (17205,)


# Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble AdaBoost classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Print the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier onely, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [15]:
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
rf_model  = BalancedRandomForestClassifier(n_estimators=100, random_state=78)

# fit the training data to the model
rf_model.fit(X_train, y_train)

BalancedRandomForestClassifier(random_state=78)

In [16]:
# make prediction using testing data
y_pred = rf_model.predict(X_test)
# Calculated the balanced accuracy score
accuracy = balanced_accuracy_score(y_test, y_pred)
print(f"accuracy (%): {accuracy*100}")

accuracy (%): 74.02706448147154


In [17]:
# Display the confusion matrix
cm = confusion_matrix(y_test, y_pred)
cm

array([[   64,    37],
       [ 2619, 14485]], dtype=int64)

In [18]:
#convert to dataframe
cm_df = pd.DataFrame(cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])
cm_df

,Predicted 0,Predicted 1
Actual 0,64,37
Actual 1,2619,14485


In [19]:
# Print the imbalanced classification report
classification_report = classification_report_imbalanced(y_test, y_pred)
print(classification_report)

                   pre       rec       spe        f1       geo       iba       sup

          0       0.02      0.63      0.85      0.05      0.73      0.53       101
          1       1.00      0.85      0.63      0.92      0.73      0.55     17104

avg / total       0.99      0.85      0.63      0.91      0.73      0.55     17205



In [56]:
# List the features sorted in descending order by feature importance
feat_importance = list(sorted(rf_model.feature_importances_, reverse = True))
feat_importance

[0.07585472539734266,
 0.06230524408109955,
 0.05667965818264351,
 0.05663850198433948,
 0.05371562622278306,
 0.03340148503501777,
 0.022628247450778845,
 0.01985176861542124,
 0.017902889655555705,
 0.017480481884896072,
 0.01610380222376191,
 0.016036774242943985,
 0.015886465752219955,
 0.015797701203935092,
 0.015578411691174344,
 0.015444658505033786,
 0.015436334235815205,
 0.01492851673178166,
 0.01485854643229977,
 0.014712218276830714,
 0.01468170591723266,
 0.014680202222172223,
 0.014671324577253396,
 0.014182724588499755,
 0.01363847122983441,
 0.013614885373992261,
 0.01319385439523231,
 0.01297625300385376,
 0.01279624556215272,
 0.012670080758343541,
 0.01221838616387114,
 0.012081125040062368,
 0.011850629309750735,
 0.011566840500256246,
 0.011083984476866595,
 0.010394977114323689,
 0.010147699841322109,
 0.010044699740973335,
 0.009371345929364505,
 0.009096891787709708,
 0.009088303568883387,
 0.00873599622191815,
 0.008462448011067715,
 0.008237622542861901,
 0.00

In [57]:
# Make a dictionary showing the column names as keys and the feature importance as values
feat_importance_dict = dict(zip(feat_importance, df2))
feat_importance_dict

{0.07585472539734266: 'loan_amnt',
 0.06230524408109955: 'int_rate',
 0.05667965818264351: 'installment',
 0.05663850198433948: 'home_ownership',
 0.05371562622278306: 'annual_inc',
 0.03340148503501777: 'verification_status',
 0.022628247450778845: 'issue_d',
 0.01985176861542124: 'loan_status',
 0.017902889655555705: 'pymnt_plan',
 0.017480481884896072: 'dti',
 0.01610380222376191: 'delinq_2yrs',
 0.016036774242943985: 'inq_last_6mths',
 0.015886465752219955: 'open_acc',
 0.015797701203935092: 'pub_rec',
 0.015578411691174344: 'revol_bal',
 0.015444658505033786: 'total_acc',
 0.015436334235815205: 'initial_list_status',
 0.01492851673178166: 'out_prncp',
 0.01485854643229977: 'out_prncp_inv',
 0.014712218276830714: 'total_pymnt',
 0.01468170591723266: 'total_pymnt_inv',
 0.014680202222172223: 'total_rec_prncp',
 0.014671324577253396: 'total_rec_int',
 0.014182724588499755: 'total_rec_late_fee',
 0.01363847122983441: 'recoveries',
 0.013614885373992261: 'collection_recovery_fee',
 0.0

In [95]:
# get the top 5 most important features from the feat_importance_dict dictionary 

feat_importance_names = list(feat_importance_dict.values())[:5]

feat_importance_values = feat_importance[:5]
                              
top_five_feat={}  

k = 0

for i in feat_importance_names:
    
        top_five_feat[i] = feat_importance_values[k]
        
        k = k+1
print(top_five_feat)

{'loan_amnt': 0.07585472539734266, 'int_rate': 0.06230524408109955, 'installment': 0.05667965818264351, 'home_ownership': 0.05663850198433948, 'annual_inc': 0.05371562622278306}


In [96]:
feat_importance_keys

['loan_amnt', 'int_rate', 'installment', 'home_ownership', 'annual_inc']

In [97]:
feat_importance_values

[0.07585472539734266,
 0.06230524408109955,
 0.05667965818264351,
 0.05663850198433948,
 0.05371562622278306]

In [98]:
# make pandas dataframe with top 5 features
top_five_df = pd.DataFrame(top_five_feat.items(), columns = ["Feature Name", "Feature Importance"])
top_five_df

,Feature Name,Feature Importance
0,loan_amnt,0.075855
1,int_rate,0.062305
2,installment,0.056680
3,home_ownership,0.056639
4,annual_inc,0.053716


### Easy Ensemble AdaBoost Classifier

In [ ]:
# Train the EasyEnsembleClassifier
from imblearn.ensemble import EasyEnsembleClassifier
eec_model = EasyEnsembleClassifier(n_estimators = 100, random_state = 78)

# fit the training data to the model
eec_model.fit(X_train, y_train)

In [ ]:
# make prediction using testing data
y_pred = eec_model.predict(X_test)
# Calculated the balanced accuracy score
accuracy = balanced_accuracy_score(y_test, y_pred)
print(f"accuracy (%): {accuracy*100}")

In [ ]:
# Display the confusion matrix
cm = confusion_matrix(y_test, y_pred)
cm

In [ ]:
#convert to dataframe
cm_df = pd.DataFrame(cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])
cm_df

In [ ]:
# Print the imbalanced classification report
classification_report = classification_report_imbalanced(y_test, y_pred)
print(classification_report)